<a href="https://colab.research.google.com/github/MicDso/-Machine-Learning-with-Python-Certification-Projects/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.0/590.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.7/579.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Fou

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-03-27 14:04:00--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-03-27 14:04:00 (10.4 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-03-27 14:04:00--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
train_dataset, valid_dataset = pd.read_csv('train-data.tsv',sep='\t',header=None,names=['class', 'text']) , pd.read_csv('valid-data.tsv',sep='\t',header=None,names=['class', 'text'])
x = keras.datasets.imdb.get_word_index(path="imdb_word_index.json")
def preprocess(df):
    df['class'] = df['class'].map({'ham': 0, 'spam': 1})
    labels = df.pop('class').astype(np.float32)
    data = df['text'].tolist()
    return data, labels
train_data, train_labels = preprocess(train_dataset)
valid_data, valid_labels = preprocess(valid_dataset)
def encode_text(text):
    tokens = [x.get(word, 0) for word in keras.preprocessing.text.text_to_word_sequence(text)]
    padded_sequence = keras.preprocessing.sequence.pad_sequences([tokens], maxlen=1000, padding='post', truncating='post')
    return np.squeeze(padded_sequence)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=100000,output_dim=64,mask_zero=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(0.01),metrics=['accuracy'])
model.fit(x=np.array([encode_text(text) for text in train_data]),y= train_labels, epochs=8, validation_data=(np.array([encode_text(text) for text in valid_data]),valid_labels))

Epoch 1/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 132s 976ms/step - accuracy: 0.9098 - loss: 0.2289 - val_accuracy: 0.9792 - val_loss: 0.0815
Epoch 2/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 128s 980ms/step - accuracy: 0.9920 - loss: 0.0338 - val_accuracy: 0.9806 - val_loss: 0.0573
Epoch 3/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 128s 981ms/step - accuracy: 0.9978 - loss: 0.0108 - val_accuracy: 0.9749 - val_loss: 0.1031
Epoch 4/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 143s 993ms/step - accuracy: 0.9976 - loss: 0.0060 - val_accuracy: 0.9806 - val_loss: 0.1022
Epoch 5/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 137s 956ms/step - accuracy: 0.9991 - loss: 0.0030 - val_accuracy: 0.9806 - val_loss: 0.0963
Epoch 6/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 144s 973ms/step - accuracy: 0.9983 - loss: 0.0034 - val_accuracy: 0.9792 - val_loss: 0.1357
Epoch 7/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 128s 981ms/step - accuracy: 0.9994 - loss: 0.0019 - val_accuracy: 0.9792 - val_loss: 0.1240
Epoch 8/8
131/131 ━━━━━━━━━━━━━━━━━━━━ 142s 981ms/step - accuracy: 0.9994 - loss: 0

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
    outcome = model.predict((np.array([(encode_text(pred_text))])))[0][0]
    label = "spam" if outcome > 0.5 else "ham"
    return [outcome, label]

pred_text = "wow, is your arm alright. that happened to me one time too"
prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step
[2.6607276e-07, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
You passed the challenge. Great job!
